In [21]:
import numpy as np
import tensorflow as tf
import math
import random
from os import listdir
import codecs
import nltk
import data
from gensim.models import Word2Vec

In [22]:
vocab_size = 324
d, id_to_token = data.get_data(vocab_size)
for k in id_to_token:
    if id_to_token[k] == 'eos':
        id_to_token[k] = '\n'

324


In [23]:
x, y = data.sample(d, 5)

In [38]:
hidden_size = 100
num_steps = 10
learning_rate = 0.01
num_runs_between_logging = 3000
epoch = 50000

In [25]:
Wxh = tf.Variable(tf.random_uniform([vocab_size, hidden_size])) 
Whh = tf.Variable(tf.random_uniform([hidden_size, hidden_size]))
bh = tf.Variable(tf.zeros([hidden_size]))
Why = tf.Variable(tf.random_uniform([hidden_size, vocab_size])) 
by = tf.Variable(tf.zeros([vocab_size]))
U_z = tf.Variable(tf.random_uniform([vocab_size, hidden_size]))
W_z = tf.Variable(tf.random_uniform([hidden_size, hidden_size]))
U_r = tf.Variable(tf.random_uniform([vocab_size, hidden_size]))
W_r = tf.Variable(tf.random_uniform([hidden_size, hidden_size]))

In [26]:
inputs = [tf.placeholder(tf.float32, [1, vocab_size]) for _ in xrange(num_steps)]
keep_prob = tf.placeholder(tf.float32)
outputs = {}
hs = {}
hs[-1] = tf.placeholder(tf.float32, [1, hidden_size])
# GRU UNIT
for i in range(len(inputs)):
    r = tf.nn.softmax(tf.matmul(inputs[i], U_r) + tf.matmul(hs[i-1], W_r))
    z = tf.nn.softmax(tf.matmul(inputs[i], U_z) + tf.matmul(hs[i-1], W_z))
    h = tf.nn.tanh(tf.matmul(inputs[i], Wxh) + tf.matmul(hs[i - 1] * r, Whh))
    hs[i] = tf.mul(tf.sub(tf.ones_like(z), z), h) + tf.mul(z , hs[i - 1])
#     hs[i]      = tf.nn.tanh(tf.matmul(inputs[i], Wxh) + tf.matmul(hs[i-1], Whh) + bh)
    hs_drop = tf.nn.dropout(hs[i], keep_prob)
    outputs[i] = tf.nn.softmax(tf.matmul(hs_drop, Why) + by)
print inputs[0], inputs[1], hs[-1], hs[0]

Tensor("Placeholder_22:0", shape=(1, 324), dtype=float32) Tensor("Placeholder_23:0", shape=(1, 324), dtype=float32) Tensor("Placeholder_33:0", shape=(1, 100), dtype=float32) Tensor("add_53:0", shape=(1, 100), dtype=float32)


In [27]:
truths = [tf.placeholder(tf.float32, [None, vocab_size]) for i in xrange(num_steps)]
cross_entropies = [-tf.reduce_sum(truths[i]*tf.log(outputs[i])) for i in xrange(num_steps)]  # These operations act element-wise.
cross_entropy = tf.add_n(cross_entropies)

In [28]:
train_step = tf.train.AdagradOptimizer(learning_rate).minimize(cross_entropy)

In [29]:
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

In [32]:
def hallucinate(sess, seed_ix, num_chars):
    ix = seed_ix
    ixes = [ix]
    hallucination_h_state = np.zeros([1, hidden_size]) 
    for i in range(num_chars):
        # Create a random starting letter 
        x_in = [[0] * vocab_size]
        x_in[0][ix] = 1
#         make_hot_vec([1,vocab_size], ix)

        feed_dict={inputs[0]: x_in}
        feed_dict.update({hs[-1]:hallucination_h_state})
        feed_dict.update({keep_prob: 1.0})

        output, hallucination_h_state = sess.run((outputs[0],hs[0]), feed_dict=feed_dict)
        probs = output[0]
        ix = np.random.choice(range(len(probs)), p=probs)
        ixes.append(ix)
    hallucination=' '.join([id_to_token[ix] for ix in ixes])
    print hallucination

In [36]:
losses = []
iterations = 0
ix = 0
h_state = np.zeros([1, hidden_size])
smooth_loss = -np.log(1.0/vocab_size)*num_steps # loss at iteration 0

In [94]:
def RunModel():
    global ix,losses,iterations,h_state,smooth_loss
    run = 0
    while True:
        letters=[]
        if run > epoch:
            break
        x, y = data.sample(d, num_steps)
        feed_dict={inputs[i]: x[i].reshape(1, -1) for i in range(num_steps)}
        feed_dict.update({truths[i]: y[i].reshape(1, -1) for i in range(num_steps)})
        feed_dict.update({hs[-1]:h_state})
        feed_dict.update({keep_prob: 0.8})

        _, h_state, loss_out = sess.run((train_step, hs[num_steps-1], cross_entropy), feed_dict=feed_dict)
        smooth_loss = smooth_loss * 0.999 + loss_out * 0.001
        ix += num_steps
        #        letters.append(data[ix])
        run += 1
        if run % num_runs_between_logging == 0:   
            iterations += 1
            #        print "train: ", ''.join(letters)
            print "========== Iteration ", run, " Loss: ", smooth_loss, " =============="
            losses.append(smooth_loss)
            hallucinate(sess, random.randint(0,vocab_size-1), 60)
RunModel()

========== Iteration  3000  Loss:  10.6051570016  ==============
tasting : 
 Acorns ripe down-pattering , 
 Blushing through the mesh 
 Thou shalt see —send her ! 
 Pleasure never is muffled , 
 One would hear so very oft ? 
 Where 's the rooks , 
 And such joys as Hebe 's the early April lark , and send abroad , 
 From the mist and dew
========== Iteration  6000  Loss:  10.3353415531  ==============
With a touch sweet Pleasure melteth 
 Through the Noon 
 And Jove grew languid.—Break the hawthorn-tree , 
 And thou shalt quaff it : What do then ? 
 behold 
 Distant harvest-carols clear ; 
 Thee a how to chide ; 
 From the mid-May ; 
 She will mix these pleasures up 
 Shaded hyacinth , however soft
========== Iteration  9000  Loss:  10.4647220807  ==============
frown and self-overaw 'd lillies , and send abroad , 
 And such joys as does its skin ; 
 Sweet birds antheming the morn : 
 Hedge-grown primrose that hath burst ; 
 'T is the early April lark , 
 While the buds and bells of May

In [98]:
hallucinate(sess, random.randint(0,vocab_size-1), 70)

waist and 
 Thou shalt see 
 And thou shalt quaff it : What do then ? 
 Fancy , 
 Or the caked snow is ever new ? 
 Open wide the mind 's cage-door , 
 She 'll dart forth , and down delights of May , sweet Fancy ! let her mossy nest ; 
 Freckled nest-eggs thou shalt hear 
 With a dark conspiracy 
 Fell the
